Phone contract choice example
================

This is a small example of using MILP to choose between options. Also see the spreadsheet solution of the problem.

Briefly, we must choose a cell phone contract from one of three providers. A contract costs a certain amount and supplies a certain number of "free minutes". We only pay for minutes above this "free" amount. Some providers also supply bundles. If we have a contract from that provier, we can add a bundle. To keep things simple in this problem, we will pretend that we can only buy one bundle (or choose not to buy a bundle at all). The problem is to minimise total cost for a known amount of minutes used.

We'll use [PuLP](https://pythonhosted.org/PuLP/) to solve this problem. The website has some good examples to get you going.

In [1]:
import pulp

We create a LpProblem instance.

In [2]:
p = pulp.LpProblem("Phone contract", pulp.LpMinimize)

Specify how many minutes to use.

In [3]:
minutes_used = 299

Define the data for the problem. Providers:

In [4]:
providers = ['MTN', 'Vodacom', 'CellC']
contract_cost = dict(zip(providers, [300, 200, 100]))
contract_minutes = dict(zip(providers, [200, 250, 150]))
contract_call_cost = dict(zip(providers, [1, 2, 3]))

We can pay extra for some "free" minutes

In [5]:
bundles = ['MTN 1', 'MTN 2', 'Vodacom']
bundle_cost = dict(zip(bundles, [20, 30, 50]))
bundle_minutes = dict(zip(bundles, [25, 40, 30]))

Define the variables used - these will all be determined by the solver. First, binary variables to decide which providers and bundles to choose.

In [6]:
contract_chosen = pulp.LpVariable.dicts('contract_chosen', providers, cat='Binary')
bundle_chosen = pulp.LpVariable.dicts('bundle_chosen', bundles, cat='Binary')

Then continuous variables to figure out some internal numbers - we'll keep track of how many minutes we have to pay for and how many minutes we'll use from the bundles.

In [7]:
minutes_paid_for = pulp.LpVariable.dicts('minutes_paid_for', providers, lowBound=0)
minutes_from_bundles = pulp.LpVariable.dicts('minutes_from_bundles', providers, lowBound=0)

In [8]:
minutes_after_free = {provider: max(minutes_used - contract_minutes[provider], 0) for provider in providers}

Objective function
-----------------

Our objective is to minimize total cost

In [9]:
p += (sum(contract_chosen[provider]*contract_cost[provider] for provider in providers) +
      sum(bundle_chosen[bundle]*bundle_cost[bundle] for bundle in bundles) +
      sum(minutes_paid_for[provider]*contract_call_cost[provider] for provider in providers))

Constraints
-----------


Must choose exactly one provider

In [10]:
p += sum(contract_chosen[provider] for provider in providers) == 1

This equality constraint can be seen as a "calculation", we're figuring out how many minutes are supplied by the bundles we've chosen. Note that I've used a naming convention here to find out which bundle belongs to which provider.

In [11]:
for provider in providers:
    p += minutes_from_bundles[provider] == sum(bundle_chosen[bundle]*bundle_minutes[bundle] 
                                               for bundle in bundles 
                                               if bundle.startswith(provider))

This constraint will figure out how many minutes need to be payed for: note that if the RHS is negative, the minutes paid for will be pushed to zero by the optimisation. The final term is a "Big M" constraint relaxation, effectively switching off this constraint for providers which have not been chosen. This can be seen as a logic statement "If provider x is chosen, we must pay for these minutes, else the constraint is so low that only the zero lower bound holds"

In [12]:
for provider in providers:
    p += minutes_paid_for[provider] >= (minutes_after_free[provider] - 
                                        minutes_from_bundles[provider] -
                                        (1-contract_chosen[provider])*minutes_used)

In [13]:
p.solve()

1

In [16]:
for provider in providers:
    if contract_chosen[provider].value():
        print('Provider:', provider)
for bundle in bundles:
    if bundle_chosen[bundle].value():
        print('Bundle:', bundle)

Provider: Vodacom
Bundle: Vodacom


In [19]:
cost = p.objective.value()
cost

288.0